In [8]:
import requests
from bs4 import BeautifulSoup

url = "https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/9149/Spain-LaLiga"  
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find("div", class_="table_container is_setup")

print(table)
# first_row = table.find('tbody')
# first_row = table.find('tbody').find('tr', {'data-row': '0'})


# data = {}

# # 행의 모든 'td'를 반복하면서 'data-stat'을 키로, 내용(text)을 값으로 저장
# for td in first_row.find_all('td'):
#     data_stat = td.get('data-stat')
#     value = td.text
#     data[data_stat] = value

None


In [11]:
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select 

url = 'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/9149/Spain-LaLiga'
browser = webdriver.Chrome()
browser.get(url)

select = Select(browser.find_element(By.ID,'seasons'))
select.select_by_visible_text("2022/2023")

table = browser.find_element(By.ID, 'standings-21073-content')
rows = table.find_elements(By.TAG_NAME, 'tr')
print(len(rows))
url_list = []

dataset = []
for row in rows:
    team = row.find_element(By.CLASS_NAME, "team-link")
    team_name = team.text
    team_url = team.get_attribute('href')
    url_list.append(team_url)

    num_matches = row.find_element(By.CLASS_NAME, "p").text
    num_win = row.find_element(By.CLASS_NAME, 'w').text
    num_draw = row.find_element(By.CLASS_NAME, 'd').text
    num_lose = row.find_element(By.CLASS_NAME, 'l').text

    ## gf : 득점, ga : 실점, gd : 득실차, pts : 승점
    goal_for = row.find_element(By.CLASS_NAME, 'gf').text
    goal_against = row.find_element(By.CLASS_NAME, 'ga').text
    goal_difference = row.find_element(By.CLASS_NAME, 'gd').text
    points = row.find_element(By.CLASS_NAME, 'pts').text

    print(team_name, num_matches, num_win, num_draw, num_lose, goal_for, goal_against, goal_difference, points, team_url)
    dataset.append([team_name, num_matches, num_win, num_draw, num_lose, goal_for, goal_against, goal_difference, points, team_url])

df = pd.DataFrame(dataset, columns=['Team', 'Matches', 'Win', 'Draw', 'Lose', 'GF', 'GA', 'GD', 'Points', 'URL'])
print(df.head(len(dataset)))
df.to_csv("./Bundesliga.csv")




20
Barcelona 38 28 4 6 70 20 +50 88 https://1xbet.whoscored.com/Teams/65/Show/Spain-Barcelona
Real Madrid 38 24 6 8 75 36 +39 78 https://1xbet.whoscored.com/Teams/52/Show/Spain-Real-Madrid
Atletico Madrid 38 23 8 7 70 33 +37 77 https://1xbet.whoscored.com/Teams/63/Show/Spain-Atletico-Madrid
Real Sociedad 38 21 8 9 51 35 +16 71 https://1xbet.whoscored.com/Teams/68/Show/Spain-Real-Sociedad
Villarreal 38 19 7 12 59 40 +19 64 https://1xbet.whoscored.com/Teams/839/Show/Spain-Villarreal
Real Betis 38 17 9 12 46 41 +5 60 https://1xbet.whoscored.com/Teams/54/Show/Spain-Real-Betis
Osasuna 38 15 8 15 37 42 -5 53 https://1xbet.whoscored.com/Teams/131/Show/Spain-Osasuna
Athletic Club 38 14 9 15 47 43 +4 51 https://1xbet.whoscored.com/Teams/53/Show/Spain-Athletic-Club
Mallorca 38 14 8 16 37 43 -6 50 https://1xbet.whoscored.com/Teams/51/Show/Spain-Mallorca
Girona 38 13 10 15 58 55 +3 49 https://1xbet.whoscored.com/Teams/2783/Show/Spain-Girona
Rayo Vallecano 38 13 10 15 45 53 -8 49 https://1xbet.whos

In [57]:

def players(i):
    table = browser.find_element(By.ID, "top-player-stats-summary-grid")

    # 테이블 tr테그 안의 정보들을 다 가져온다.
    rows = table.find_elements(By.TAG_NAME, "tr")
    # 데이터셋 생성
    dataset = []
    for row in rows[1:]:  # 두번째 행부터 이름, 위치, 포지션 정보 들어있음
        # 처음 선수부터 td들을 전부 들고오며 확인
        cells = row.find_elements(By.TAG_NAME, "td")

        player_info = row.find_element(By.CLASS_NAME, "grid-abs")
        a_tag = player_info.find_element(By.TAG_NAME, "a")
        name = a_tag.find_element(By.TAG_NAME, "span").text.strip()
        meta_data = player_info.find_elements(By.CLASS_NAME, "player-meta-data")
        age = meta_data[0].text.strip()
        position = meta_data[1].text.lstrip(',').strip() 

        height = cells[2].text
        weight = cells[3].text
        apps = cells[4].text
        mins = cells[5].text
        goals = cells[6].text
        assists = cells[7].text
        yellow_card = cells[8].text
        red_card = cells[9].text
        shots_per_game = cells[10].text
        pass_success = cells[11].text
        aerials_won = cells[12].text
        man_of_the_match = cells[13].text
        rating = cells[14].text
        
   
        dataset.append([
            name, position, age, height, weight, apps, mins, goals, assists, yellow_card, red_card,
            shots_per_game, pass_success, aerials_won, man_of_the_match, rating
        ])
    

    team = browser.find_element(By.CLASS_NAME, "team-header-name")

    df = pd.DataFrame(
        dataset,
        columns=[
            "Player", "Position", "Age", "Height", "Weight", "Apps", "Mins", "Goals", "Assists", "Yel", "Red",
            "SpG", "PS%", "AerialsWon", "MotM", "Rating"
        ]
    )
    # CSV 파일로 저장
    df.to_csv(f"{team}_player_stats.csv", index=False)

    # DataFrame 출력
    print(df.head(len(dataset)))

i = 0
for url in url_list:
    browser = webdriver.Chrome()
    browser.get(url)
    i +=1
    players(i)
    break





height 


In [51]:
%cd C:\Users\josuh\Desktop\부캠\프로젝트\EDA

c:\Users\josuh\Desktop\부캠\프로젝트\EDA


In [53]:
df = pd.read_csv('Espanyol_player_stats.csv')
df

,Player,Position,Age,Height,Weight,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Rating
0,Gaston Valles,NaN,0,NaN,0,0,1(7),NaN,-,NaN,1,-,NaN,NaN,NaN,NaN
1,Ian Forns,NaN,0,NaN,0,0,2,NaN,-,NaN,-,-,NaN,NaN,NaN,NaN
2,Nico Ribaudo,forward,23,NaN,171,67,15(4),NaN,3,NaN,4,-,NaN,NaN,NaN,NaN
3,Omar Sadik,NaN,0,NaN,0,0,2(1),NaN,-,NaN,1,-,NaN,NaN,NaN,NaN
4,Ramon,forward,23,NaN,179,70,3(10),NaN,-,NaN,2,-,NaN,NaN,NaN,NaN
5,Ruben Sanchez,defender,23,NaN,185,73,1(3),NaN,-,NaN,-,-,NaN,NaN,NaN,NaN
6,Álvaro Aguado,M(C),28,NaN,174,66,20(7),NaN,1,NaN,1,1,NaN,NaN,NaN,NaN
7,Antoniu Roca,Forward,21,NaN,0,0,0(2),NaN,-,NaN,1,-,NaN,NaN,NaN,NaN
8,Brian Oliván,D(CL),30,NaN,177,73,27(2),NaN,-,NaN,5,-,NaN,NaN,NaN,NaN
9,Edu Expósito,M(C),27,NaN,178,68,15(6),NaN,1,NaN,3,-,NaN,NaN,NaN,NaN


In [72]:
df = pd.read_csv('Laliga_Player_22_23.csv')


for team in df.team_name.unique():
    ds = df[df['team_name'] == team]
    ds.to_csv(f'{team}_Stats_22_23.csv', index=False)
    




In [60]:
df = pd.read_csv('Laliga_Value_22_23.csv')

df

,Unnamed: 0,Team,Squad,Average age,Foreigne Players,Average Market Value,Market Value
0,0,Real Madrid,36,25.4,20,€25.65m,€923.45m
1,1,FC Barcelona,41,24.0,14,€20.41m,€836.80m
2,2,Atlético de Madrid,40,25.4,22,€13.36m,€534.30m
3,3,Real Sociedad,37,24.5,6,€11.94m,€441.70m
4,4,Villarreal CF,43,25.8,15,€8.45m,€363.50m
5,5,Valencia CF,40,24.0,18,€6.24m,€249.78m
6,6,Real Betis Balompié,38,27.1,13,€6.48m,€246.40m
7,7,Sevilla FC,42,26.3,23,€5.54m,€232.60m
8,8,Athletic Bilbao,31,26.6,1,€7.18m,€222.45m
9,9,CA Osasuna,31,26.7,4,€4.84m,€150.05m


In [80]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

Headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'}
url = 'https://www.transfermarkt.com/real-madrid/kader/verein/418/plus/0/galerie/0?saison_id=2022'
r = requests.get(url, headers=Headers)

soup =bs(r.text, 'html.parser')
player_info = soup.find_all('tr', class_=['odd','even'])

player_data = []
for player in player_info:

    name = player.find('td', class_='hauptlink').find('a').text.strip()
    position = player.find('td', class_='posrela').find_all('td')[-1].text.strip()

    dob_age_text = player.find_all('td', class_='zentriert')[1].text.strip() 
    dob = ' '.join(dob_age_text.split()[:-1])  
    age = dob_age_text.split()[-1].strip('()') 
    
    nationality = player.find('img', class_='flaggenrahmen')['title']
    market_value = player.find('td', class_='rechts hauptlink').text.strip()

    player_data.append({
        'Name': name,
        'Position': position,
        'Date of Birth': dob,
        'Age': age,
        'Nationality': nationality,
        'Market Value': market_value
    })

player_data = pd.DataFrame(player_data)
player_data.to_csv('Inter_Player_Value.csv', index=False)
player_data

,Name,Position,Date of Birth,Age,Nationality,Market Value
0,Thibaut Courtois,Goalkeeper,,31,Belgium,€45.00m
1,Andriy Lunin,Goalkeeper,,24,Ukraine,€5.00m
2,Mario de Luis,Goalkeeper,,21,Spain,€500k
3,Luis López,Goalkeeper,,22,Spain,€300k
4,Lucas Cañizares,Goalkeeper,,21,Spain,€200k
5,Diego Piñeiro,Goalkeeper,,19,Spain,-
6,Fran González,Goalkeeper,,18,Spain,-
7,Éder Militão,Centre-Back,,25,Brazil,€70.00m
8,David Alaba,Centre-Back,,31,Austria,€40.00m
9,Antonio Rüdiger,Centre-Back,,30,Germany,€32.00m


## K league

In [19]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

Headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'}
url = 'https://fbref.com/en/comps/55/2023/stats/2023-K-League-1-Stats'
r = requests.get(url, headers=Headers)

soup =bs(r.text, 'html.parser')
table = soup.find('div', class_ = "div_stats_standard")
player_info = table.find_all('tr')

columns = [
            "ranker", "player", "nationality", "position", "team", "age",
            "birth_year", "games", "games_starts", "minutes", "minutes_90s",
            "goals", "assists", "goals_assists", "goals_pens", "pens_made",
            "pens_att", "cards_yellow", "cards_red", "goals_per90", "assists_per90",
            "goals_assists_per90", "goals_pens_per90", "goals_assists_pens_per90",
            "matches"
        ]


player_data = []
for player in player_info:
    data = []
    for info in player.find_all(["th", "td"]):
        if info.find("a"):
            data.append(info.find("a").text.strip())
        else:
            data.append(info.text.strip())
        
    player_dict = {columns[i]: data[i] for i in range(len(columns))}
        
    player_data.append(player_dict)

df = pd.DataFrame(player_data)
df
    

# container = driver.find_element(By.ID, 'stats_standard')
# container_tmp = container.find_element(By.TAG_NAME, 'tbody') 
# player_container = container_tmp.find_elements(By.TAG_NAME, 'tr') # 각각의 선수 데이터를 모두 가져와서 리스트에 저장
# all_data = []
    

AttributeError: 'NoneType' object has no attribute 'find_all'